In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
import torch

/Users/andrewly/opt/anaconda3/envs/frac-attn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [126]:
tokenizer_src = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-de-en')  # German tokenizer
tokenizer_trg = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')  # English tokenizer

/Users/andrewly/opt/anaconda3/envs/frac-attn/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/andrewly/opt/anaconda3/envs/frac-attn/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [85]:
def preprocess_function(examples, tokenizer_src, src_language, trg_language, max_length):
    inputs = [example[src_language] for example in examples["translation"]]
    targets = [example[trg_language] for example in examples["translation"]]
    model_inputs = tokenizer_src(inputs, text_target=targets, max_length=max_length, truncation=True, padding="max_length")
    labels = tokenizer_trg(targets, max_length=max_length, truncation=True, padding="max_length")["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

def prepare_data(tokenizer_src, tokenizer_trg, batch_size=4, num_workers=2, test_fraction=0.2, max_length=512):
    # Load dataset; ignore validation set (tst2013) and use test set only (tst2014)
    src_language, trg_language = 'de', 'en'
    dataset = load_dataset("ted_talks_iwslt", language_pair=(src_language, trg_language), year="2014")
    dataset = dataset.train_test_split(test_size=test_fraction, shuffle=True)
    trainset, testset = dataset['train'], dataset['test']
    # Preprocess datasets
    tokenized_trainset = trainset.map(lambda examples: preprocess_function(examples, tokenizer_src, src_language, tokenizer_trg, trg_language, max_length), batched=True)
    tokenized_testset = testset.map(lambda examples: preprocess_function(examples, tokenizer_src, src_language, tokenizer_trg, trg_language, max_length), batched=True)
    # Create dataloaders
    trainloader = torch.utils.data.DataLoader(tokenized_trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(tokenized_testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return trainloader, testloader

In [6]:
dataset = load_dataset("ted_talks_iwslt", language_pair=("de", "en"), year="2014")

/Users/andrewly/opt/anaconda3/envs/frac-attn/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for ted_talks_iwslt contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ted_talks_iwslt
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 2972 examples [00:06, 461.77 examples/s]


In [11]:
dataset = dataset["train"].train_test_split(test_size=0.1, shuffle=True)

In [20]:
testset = dataset["test"]

In [86]:
tokenized_testset = testset.map(lambda examples: preprocess_function(examples, tokenizer_src, "de", "en", 128), batched=True)

Map: 100%|██████████| 298/298 [00:00<00:00, 2442.14 examples/s]


In [92]:
testloader = torch.utils.data.DataLoader(tokenized_testset, batch_size=2, shuffle=False, num_workers=1)

In [93]:
batch_size = 2
ix = torch.randint(len(testloader), (batch_size,))
x = torch.stack([data.dataset[i][0] for i in ix])
y = torch.tensor([data.dataset[i][1] for i in ix])

In [114]:
config = {
    "beta": 1,
    "bandwidth": 1,
    "sphere_radius": 1,
    "hidden_size": 1,
    "num_encoder_layers": 1,
    "num_decoder_layers": 1,
    "num_attention_heads": 1,
    "intermediate_size": 4, # 4 * hidden_size
    "hidden_dropout_prob": 0,
    "encoder_dropout_prob": 0,
    "decoder_dropout_prob": 0,
    "attention_probs_dropout_prob": 0,
    "initializer_range": 0.1,
    "qkv_bias": True,
    "use_faster_attention": True,
    "src_vocab_size": tokenizer_src.vocab_size,
    "src_pad_token_id": tokenizer_src.pad_token_id,
    "trg_vocab_size": tokenizer_trg.vocab_size,
    "trg_pad_token_id": tokenizer_trg.pad_token_id,
    "max_length": 128,
}
model = FNSForTranslation(config)    

In [115]:
model 

FNSForTranslation(
  (encoder): FNSEncoder(
    (token_embedding): Embedding(58101, 1, padding_idx=58100)
    (positional_embedding): Embedding(128, 1)
    (dropout): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0): FNSEncoderBlock(
        (attention): FasterFNSMultiHeadAttention(
          (qkv_projection): Linear(in_features=1, out_features=3, bias=True)
          (attn_dropout): Dropout(p=0, inplace=False)
          (output_projection): Linear(in_features=1, out_features=1, bias=True)
          (output_dropout): Dropout(p=0, inplace=False)
        )
        (layernorm_1): LayerNorm((1,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (dense_1): Linear(in_features=1, out_features=4, bias=True)
          (activation): NewGELUActivation()
          (dense_2): Linear(in_features=4, out_features=1, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )
        (layernorm_2): LayerNorm((1,), eps=1e-05, elementwise_affine=True)
      )
   

In [116]:
def greedy_decode(model, source, tokenizer_trg, max_len, device):
    bos_idx = tokenizer_trg.bos_token_id
    eos_idx = tokenizer_trg.eos_token_id

    # Precompute the encoder output and reuse it for every step
    encoder_output = model.encode(source)
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(bos_idx).type_as(source).to(device)
    while True:
        if decoder_input.size(1) == max_len:
            break

        # build causal mask for target 
        decoder_mask = torch.triu(torch.ones((1, decoder_input.size(1), decoder_input.size(1))), diagonal=1).type(torch.int)
        decoder_mask = (decoder_mask == 0).type_as(source_mask).to(device)

        # calculate output
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # get next token
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)